# CSV파일을 이용하여 유튜브에서 음원추출

In [ ]:
!pip install - pytube
!pip install -q pycaret

In [ ]:
from pytube import YouTube
import time
import os
import librosa, librosa.display
import numpy as np
import pandas as pd
from scipy.io import wavfile
import shutil
import soundfile as sf
import sklearn
import random
from pycaret.classification import *
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.preprocessing import StandardScaler,MinMaxScaler



In [ ]:
import torch

In [ ]:
import tensorflow

In [ ]:
torch.__version__

'2.3.0+cu121'

In [ ]:
sklearn.__version__

'1.2.2'

In [ ]:
# csv 파일 불러오기
def load_music(dir_path):
    dir_path = dir_path
    global music_path
    music_path = list()
    for (root,directories, files) in os.walk(dir_path):
        for file in files:
            music_path.append(os.path.join(root, file))
    music_path.sort()
load_music('/content/drive/MyDrive/dataton/csv수집/new_csv')

In [ ]:
# 음악 저장 폴더 생성 없을 시 실행
# def make_folder():
#     for i in range(len(music_path)):
#         os.mkdir(f"/content/drive/MyDrive/dataton/csv수집/new_music/{music_path[i].split('/')[-1].split('.')[0]}")
# make_folder()

In [ ]:
# 음악 저장 경로
def load_folder(dir_path):
    dir_path = dir_path
    global file_path
    file_path = list()

    for (root,directories, files) in os.walk(dir_path):
        for directorie in directories:
            file_path.append(os.path.join(root, directorie))
    file_path.sort()
load_folder('/content/drive/MyDrive/dataton/csv수집/new_music')


In [ ]:
# mp3 파일을 wav 파일 30초로 변환
def split_and_save(wav_path, output_dir, segment_length=30):

    """
    - wav_path : 로드할 WAV 파일 또는 MP3 파일의 경로
    - output_dir : WAV 파일들을 저장할 디렉토리 경로
    - segment_length : 분할할 세그먼트의 길이 (초 단위, 기본값은 30초)
    """

    # 출력 디렉토리가 존재하지 않으면 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    y, sr = librosa.load(wav_path, sr=None)
    # 샘플의 총 길이 계산
    total_samples = len(y)
    # 전체 오디오를 segment_length 초 단위로 나눈 각 세그먼트의 샘플 수 계산
    output_path = f"{output_dir}/segment_{i}.wav"
    sf.write(output_path, y[0:sr*120], sr)

In [ ]:
# 유툽에서 mp4 파일 다운
def download_music(file_path,music_path):
    global i
    df = pd.read_csv(file_path)

    df = df[['youtube_url']].dropna(axis = 0).reset_index(drop = True)
    url = df['youtube_url'].to_list()
    word = 'youtube'
    ben = [i for i in range(len(url)) if word not in url[i]]
    for i in ben:
        del url[i]

    i = 0
    while i < (len(url)):
      yt = YouTube(url[i])

      i +=1
      name = str(i) + ".mp4"
      try:
        music = yt.streams.filter(only_audio=True, audio_codec="mp4a.40.5" if True else "mp4a.40.2").first()
        music.download(output_path=music_path, filename=name)
        os.rename(f'{music_path}/{name}',f'{music_path}/{i}.mp3')
        # os.remove(f'{file_path}/{name}')
        split_and_save(f'{music_path}/{str(i)}.mp3', music_path, segment_length=30)
      except:
        print(f'{i}번째 오류')

In [ ]:
# 다운로드
def full_download():
    load_music()
    load_folder()
    # print(file_path[1].split('/')[-1])
    # download_music(music_path[1],file_path[1])
    for i in range(len(file_path)):
        print(file_path[i].split('/')[-1])
        download_music(music_path[i],file_path[i])

In [ ]:
%%time
full_download()

# wav파일에서 피처 추출

## 스펙트럴

In [ ]:
%%time
# 데이터 가져오기(폴더 정보)
def load_path():
    dir_path = '/content/drive/MyDrive/dataton/csv수집/2minute'
    global file_path
    file_path = list()
    for (root,directories, files) in os.walk(dir_path):
        for directorie in directories:
            file_path.append(os.path.join(root, directorie))
    file_path.sort()
# 데이터 가져오기(파일 정보)
def load_music(file_path):
    dir_path = file_path
    global music_path
    music_path = list()
    for (root,directories, files) in os.walk(dir_path):
        for file in files:
            music_path.append(os.path.join(root, file))
    music_path.sort()

# 리스트 생성
def make_list_spec():
    global rolloff_mean, rolloff_var, centroid_mean, centroid_var,flux_mean,flux_var, contrast_mean, contrast_var, flatness_mean,flatness_var,bw_mean,bw_var,chroma_mean, chroma_var,\
    mfcc1_mean, mfcc1_var,mfcc2_mean,mfcc2_var,mfcc3_mean, mfcc3_var,mfcc4_mean, mfcc4_var,mfcc5_mean, mfcc5_var,mfcc6_mean, \
    mfcc6_var,mfcc7_mean, mfcc7_var,mfcc8_mean, mfcc8_var,mfcc9_mean, mfcc9_var,mfcc10_mean, mfcc10_var,mfcc11_mean, mfcc11_var,mfcc12_mean, mfcc12_var,mfcc13_mean, \
    mfcc13_var,mfcc14_mean, mfcc14_var,mfcc15_mean, mfcc15_var,mfcc16_mean, mfcc16_var,mfcc17_mean, mfcc17_var,mfcc18_mean, mfcc18_var,mfcc19_mean, mfcc19_var,mfcc20_mean, mfcc20_var,\
    label, name
    rolloff_mean = list()
    rolloff_var = list()
    centroid_mean = list()
    centroid_var = list()
    flux_mean = list()
    flux_var = list()
    contrast_mean = list()
    contrast_var = list()
    flatness_mean = list()
    flatness_var = list()
    bw_mean = list()
    bw_var = list()
    chroma_mean = list()
    chroma_var = list()
    label = list()
    name = list()
    mfcc1_mean = list()
    mfcc1_var = list()
    mfcc2_mean = list()
    mfcc2_var = list()
    mfcc3_mean = list()
    mfcc3_var = list()
    mfcc4_mean = list()
    mfcc4_var = list()
    mfcc5_mean = list()
    mfcc5_var = list()
    mfcc6_mean = list()
    mfcc6_var = list()
    mfcc7_mean = list()
    mfcc7_var = list()
    mfcc8_mean = list()
    mfcc8_var = list()
    mfcc9_mean = list()
    mfcc9_var = list()
    mfcc10_mean = list()
    mfcc10_var = list()
    mfcc11_mean = list()
    mfcc11_var = list()
    mfcc12_mean = list()
    mfcc12_var = list()
    mfcc13_mean = list()
    mfcc13_var = list()
    mfcc14_mean = list()
    mfcc14_var = list()
    mfcc15_mean = list()
    mfcc15_var = list()
    mfcc16_mean = list()
    mfcc16_var = list()
    mfcc17_mean = list()
    mfcc17_var = list()
    mfcc18_mean = list()
    mfcc18_var = list()
    mfcc19_mean = list()
    mfcc19_var = list()
    mfcc20_mean = list()
    mfcc20_var = list()



# 스펙트럴 롤오프(Spectral Rolloff) : 주파수 스펙트럼의 하위 85%의 에너지를 포함하는 주파수를 나타냅니다.
def get_rolloff(y,sr):
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)[0]
    rolloff_mean.append(spectral_rolloff.mean())
    rolloff_var.append(spectral_rolloff.var())

# centroid (소리를 주파수 표현했을 때, 주파수의 가중평균을 계산하여 소리의 "무게 중심"이 어딘지를 알려주는 지표)
def get_centroid(y,sr):
    spectral_centroids = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    centroid_mean.append(spectral_centroids.mean())
    centroid_var.append(spectral_centroids.var())

# 스펙트럴 플럭스(Spectral Flux) : 스펙트럼의 변화량을 나타내며, 음향 신호의 동적인 변화를 측정합니다.
def get_flux(y,sr):
    spectral_flux = librosa.onset.onset_strength(y=y, sr=sr)
    flux_mean.append(spectral_flux.mean())
    flux_var.append(spectral_flux.var())

# 스펙트럴 콘트라스트(Spectral Contrast) : 스펙트럼의 피크와 계곡의 차이를 나타냅니다.
def get_contrast(y,sr):
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    contrast_mean.append(spectral_contrast.mean())
    contrast_var.append(spectral_contrast.var())

# Spectral Flatness : 스펙트럼이 얼마나 평탄한지를 나타내는 지표
def get_flatness(y,sr):
    D = np.abs(librosa.stft(y))
    flatness = librosa.feature.spectral_flatness(S=D)[0]
    flatness_mean.append(flatness.mean())
    flatness_var.append(flatness.var())

#  크로마 피처 (Chroma Features) 추출 : 크로마 피처는 음악의 하모닉과 멜로딕 정보를 캡처합니다.
def get_chroma(y,sr):
    chromagram = librosa.feature.chroma_stft(y=y, sr=sr).flatten()
    chroma_mean.append(chromagram.mean())
    chroma_var.append(chromagram.var())


# bandwidth (대역폭)
def get_bandwith(y,sr):
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr).flatten()
    bw_mean.append(spec_bw.mean())
    bw_var.append(spec_bw.var())


def get_mfcc(y,sr):
    mfccs = librosa.feature.mfcc(y= y, sr=sr)
    mfcc1_mean.append(mfccs[0].mean())
    mfcc1_var.append(mfccs[0].var())
    mfcc2_mean.append(mfccs[1].mean())
    mfcc2_var.append(mfccs[1].var())
    mfcc3_mean.append(mfccs[2].mean())
    mfcc3_var.append(mfccs[2].var())
    mfcc4_mean.append(mfccs[3].mean())
    mfcc4_var.append(mfccs[3].var())
    mfcc5_mean.append(mfccs[4].mean())
    mfcc5_var.append(mfccs[4].var())
    mfcc6_mean.append(mfccs[5].mean())
    mfcc6_var.append(mfccs[5].var())
    mfcc7_mean.append(mfccs[6].mean())
    mfcc7_var.append(mfccs[6].var())
    mfcc8_mean.append(mfccs[7].mean())
    mfcc8_var.append(mfccs[7].var())
    mfcc9_mean.append(mfccs[8].mean())
    mfcc9_var.append(mfccs[8].var())
    mfcc10_mean.append(mfccs[9].mean())
    mfcc10_var.append(mfccs[9].var())
    mfcc11_mean.append(mfccs[10].mean())
    mfcc11_var.append(mfccs[10].var())
    mfcc12_mean.append(mfccs[11].mean())
    mfcc12_var.append(mfccs[11].var())
    mfcc13_mean.append(mfccs[12].mean())
    mfcc13_var.append(mfccs[12].var())
    mfcc14_mean.append(mfccs[13].mean())
    mfcc14_var.append(mfccs[13].var())
    mfcc15_mean.append(mfccs[14].mean())
    mfcc15_var.append(mfccs[14].var())
    mfcc16_mean.append(mfccs[15].mean())
    mfcc16_var.append(mfccs[15].var())
    mfcc17_mean.append(mfccs[16].mean())
    mfcc17_var.append(mfccs[16].var())
    mfcc18_mean.append(mfccs[17].mean())
    mfcc18_var.append(mfccs[17].var())
    mfcc19_mean.append(mfccs[18].mean())
    mfcc19_var.append(mfccs[18].var())
    mfcc20_mean.append(mfccs[19].mean())
    mfcc20_var.append(mfccs[19].var())


# label (target(mood))
def get_label(file_path):
    mood = file_path.split('/')[-1].split('(')[0]
    label.append(mood)

# 파일명 호출
def file_name(music_path):
    names = music_path.split('/')[-1].split('_')[-1].split('.')[0]
    name.append(names)

#
def make_DataFrame():
    cnt = 0
    load_path()
    make_list_spec()
    for i in range(len(file_path)):
        print(file_path[i].split('/')[-1].split('(')[0])
        load_music(file_path[i])
        for j in range(len(music_path)):
            get_label(file_path[i])
            file_name(music_path[j])
            y, sr = librosa.load(music_path[j])
            get_rolloff(y,sr)
            get_centroid(y,sr)
            get_flux(y,sr)
            get_contrast(y,sr)
            get_mfcc(y,sr)
            get_bandwith(y,sr)
            get_chroma(y,sr)
            get_flatness(y,sr)
            cnt+=1
            if cnt % 100 == 0:
                print(cnt)

    df = pd.DataFrame({'name':name,'target':label,'rolloff_mean':rolloff_mean, 'rolloff_var':rolloff_var, 'centroid_mean':centroid_mean, 'centroid_var':centroid_var,'flux_mean':flux_mean,
                       'flux_var':flux_var, 'contrast_mean':contrast_mean, 'contrast_var':contrast_var,'flatness_mean':flatness_mean,'flatness_var': flatness_var,'bw_mean':bw_mean,'bw_var':bw_var,
                       'chroma_mean':chroma_mean, 'chroma_var':chroma_var, 'mfcc1_mean':mfcc1_mean,'mfcc1_var':mfcc1_var,'mfcc2_mean':mfcc2_mean,'mfcc2_var':mfcc2_var,'mfcc3_mean':mfcc3_mean,
                       'mfcc3_var':mfcc3_var,'mfcc4_mean':mfcc4_mean,'mfcc4_var':mfcc4_var,'mfcc5_mean':mfcc5_mean,'mfcc5_var':mfcc5_var,'mfcc6_mean':mfcc6_mean,'mfcc6_var':mfcc6_var,'mfcc7_mean':mfcc7_mean,
                       'mfcc7_var':mfcc7_var,'mfcc8_mean':mfcc8_mean,'mfcc8_var':mfcc8_var,'mfcc9_mean':mfcc9_mean,'mfcc9_var':mfcc9_var,'mfcc10_mean':mfcc10_mean,'mfcc10_var':mfcc10_var,
                       'mfcc11_mean':mfcc11_mean,'mfcc11_var':mfcc11_var,'mfcc12_mean':mfcc12_mean,'mfcc12_var':mfcc12_var,'mfcc13_mean':mfcc13_mean,'mfcc13_var':mfcc13_var,'mfcc14_mean':mfcc14_mean,
                       'mfcc14_var':mfcc14_var,'mfcc15_mean':mfcc15_mean,'mfcc15_var':mfcc15_var,'mfcc16_mean':mfcc16_mean,'mfcc16_var':mfcc16_var,'mfcc17_mean':mfcc17_mean,'mfcc17_var':mfcc17_var,
                       'mfcc18_mean':mfcc18_mean,'mfcc18_var':mfcc18_var,'mfcc19_mean':mfcc19_mean,'mfcc19_var':mfcc19_var,'mfcc20_mean':mfcc20_mean,'mfcc20_var':mfcc20_var})
    return df
a = make_DataFrame()

## 음향특징 추출

In [ ]:
%%time
# 데이터 가져오기(폴더 정보)
def load_path():
    dir_path = '/content/drive/MyDrive/dataton/csv수집/2minute'
    global file_path
    file_path = list()
    for (root,directories, files) in os.walk(dir_path):
        for directorie in directories:
            file_path.append(os.path.join(root, directorie))
    file_path.sort()
# 데이터 가져오기(파일 정보)
def load_music(file_path):
    dir_path = file_path
    global music_path
    music_path = list()
    for (root,directories, files) in os.walk(dir_path):
        for file in files:
            music_path.append(os.path.join(root, file))
    music_path.sort()

def make_list_acoustic():
    global rms_mean, rms_var, zero_crossings_mean, zero_crossings_var, tempo, label, name
    rms_mean= list()
    rms_var = list()
    zero_crossings_mean = list()
    zero_crossings_var = list()
    tempo = list()
    label = list()
    name = list()




def get_label(file_path):
    mood = file_path.split('/')[-1].split('(')[0]
    label.append(mood)

def file_name(music_path):
    names = music_path.split('/')[-1].split('_')[-1].split('.')[0]
    name.append(names)



# 루트 평균 제곱(RMS) 에너지 : RMS 에너지는 신호의 에너지를 나타내며, 신호의 진폭을 측정하는 데 사용됩니다.
def get_rms(y):
    rmse = librosa.feature.rms(y=y)[0]
    rms_mean.append(rmse.mean())
    rms_var.append(rmse.var())

#  제로 크로싱율 (Zero-Crossing Rate) : 제로 크로싱율은 신호가 0을 교차하는 횟수를 나타내며, 음의 거칠기나 노이즈를 측정하는 데 사용됩니다.
def get_zero_crossing_rate(y):
    zero_crossings = librosa.zero_crossings(y, pad=False)
    zero_crossings_mean.append(zero_crossings.mean())
    zero_crossings_var.append(zero_crossings.var())

# 음악의 템포 추출
def get_tempo(y,sr):
    tempos, _ = librosa.beat.beat_track(y=y, sr=sr)
    tempo.append(tempos[0])

# 데이터 프레임 생성
def make_DataFrame():
    cnt = 0
    load_path()
    make_list_acoustic()
    for i in range(len(file_path)):
        print(file_path[i].split('/')[-1].split('(')[0])
        load_music(file_path[i])
        for j in range(len(music_path)):
            get_label(file_path[i])
            file_name(music_path[j])
            y, sr = librosa.load(music_path[j])
            get_zero_crossing_rate(y)
            get_rms(y)
            get_tempo(y,sr)
            cnt+=1
            if cnt % 100 == 0:
                print(cnt)
    df = pd.DataFrame({'name':name, 'target':label, 'rms_mean':rms_mean, 'rms_var':rms_var, 'zero_crossings_mean':zero_crossings_mean, 'zero_crossings_var':zero_crossings_var,'tempo':tempo,})
    return df
b = make_DataFrame()

## 음악 구조적 특징 추출

In [ ]:
%%time
# 데이터 가져오기(폴더 정보)
def load_path():
    dir_path = '/content/drive/MyDrive/dataton/csv수집/2minute'
    global file_path
    file_path = list()
    for (root,directories, files) in os.walk(dir_path):
        for directorie in directories:
            file_path.append(os.path.join(root, directorie))
    file_path.sort()
# 데이터 가져오기(파일 정보)
def load_music(file_path):
    dir_path = file_path
    global music_path
    music_path = list()
    for (root,directories, files) in os.walk(dir_path):
        for file in files:
            music_path.append(os.path.join(root, file))
    music_path.sort()

def make_list_musical():
    global beat_mean, beat_var, hamonic_mean,hamonic_var,perc_mean,perc_var, label, name
    beat_mean= list()
    beat_var = list()
    zero_crossings_mean = list()
    zero_crossings_var = list()
    hamonic_mean = list()
    hamonic_var = list()
    perc_mean = list()
    perc_var = list()
    label = list()
    name = list()

def get_label(file_path):
    mood = file_path.split('/')[-1].split('(')[0]
    label.append(mood)

def file_name(music_path):
    names = music_path.split('/')[-1].split('_')[-1].split('.')[0]
    name.append(names)

# 리듬 패턴 (Rhythm Patterns) 추출 : 리듬 패턴을 추출하기 위해 오디오 신호의 주기성을 분석합니다.
def get_phythm(y,sr):
    onset_env = librosa.onset.onset_strength(y=y, sr=sr)
    tempo, beats = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    beat_times = librosa.frames_to_time(beats, sr=sr)
    beat_mean.append(beat_times.mean())
    beat_var.append(beat_times.var())

# Hamonic,Percussive_Components (hamonic : 사람의 귀로 구분할 수 없는 특징들(음악의 색깔), Percussives: 리듬과 감정을 나타내는 충격파)
def get_Hamonic_and_Percussive_Components(y):
    y_harm, y_perc = librosa.effects.hpss(y)
    hamonic_mean.append(y_harm.mean())
    hamonic_var.append(y_harm.var())
    perc_mean.append(y_perc.mean())
    perc_var.append(y_perc.var())

# 데이터 프레임 생성
def make_DataFrame():
    cnt = 0
    load_path()
    make_list_musical()
    for i in range(len(file_path)):
        print(file_path[i].split('/')[-1].split('(')[0])
        load_music(file_path[i])
        for j in range(len(music_path)):
            get_label(file_path[i])
            file_name(music_path[j])
            y, sr = librosa.load(music_path[j])
            get_phythm(y,sr)
            get_Hamonic_and_Percussive_Components(y)
            cnt+=1
            if cnt % 100 == 0:
                print(cnt)
    df = pd.DataFrame({'name':name, 'target':label,'beat_mean':beat_mean, 'beat_var':beat_var, 'hamonic_mean':hamonic_mean,'hamonic_var':hamonic_var,'perc_mean':perc_mean,'perc_var':perc_var})
    return df

c = make_DataFrame()

# 멜 스펙토그램 CNN

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the GPU")
else:
    device = torch.device("cpu")
    print("running on the CPU")

In [ ]:
CFG = {
    "SR":22050,
    "SEED":42,
    "BATCH_SIZE":32,
    "TOTAL_BATCH_SIZE":32,
    "EPOCHS":100,
    "LR":1e-4,
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
def load_path():
    dir_path = r"C:\Users\alsgu\OneDrive\바탕 화면\새 폴더"
    global file_path
    file_path = list()

    for (root,directories, files) in os.walk(dir_path):
        for directorie in directories:
            file_path.append(os.path.join(root, directorie))
    file_path.sort()
load_path()
def load_music(file_path):
    dir_path = file_path
    global music_path
    music_path = list()
    for (root,directories, files) in os.walk(dir_path):
        for file in files:
            music_path.append(os.path.join(root, file))
    music_path.sort()

In [ ]:
label = []
path = []
for i in file_path:
    load_music(i)
    len(music_path)
    for j in music_path:
        path.append(j)
        label.append(i.split("\\")[-1])

In [ ]:
df= pd.DataFrame({'path': path, 'label':label})
df

In [ ]:
df['label'].value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['path'], df['label'], test_size = 0.2, random_state=42, stratify=df['label'])

In [ ]:
train = pd.concat([X_train,y_train], axis=1).reset_index(drop = True)
test = pd.concat([X_test, y_test], axis=1).reset_index(drop = True)

In [ ]:
def train_dataset():
    dataset = []
    for i in tqdm(range(len(train['path'])),colour='green'):
        if 'wav' in train['path'][i]:
            data, sr = librosa.load(train['path'][i], sr = 22050)
            class_label = train['label'][i]
            dataset.append([data,class_label])

    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data','label'])

In [ ]:
def test_dataset():
    dataset = []
    for i in tqdm(range(len(test['path'])),colour='green'):
        if 'wav' in test['path'][i]:
            data, sr = librosa.load(test['path'][i], sr = 22050)
            class_label = test['label'][i]
            dataset.append([data,class_label])

    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data','label'])

In [ ]:
train_wav = train_dataset()
test_wav = test_dataset()

In [ ]:
train_x = np.array(train_wav.data)
test_x = np.array(test_wav.data)

In [ ]:
def get_mini(data):

    mini = 9999999
    for i in data:
        if len(i) < mini:
            mini = len(i)

    return mini

train_mini = get_mini(train_x)
test_mini = get_mini(test_x)

#음성들의 길이를 맞춰줍니다.

mini = np.min([train_mini, test_mini])

In [ ]:
def set_length(data, d_mini):

    result = []
    for i in data:
        result.append(i[:d_mini])
    result = np.array(result)

    return result

train_x = set_length(train_x, mini)
test_x = set_length(test_x, mini)

In [ ]:
print('train :', train_x.shape)
print('test :', test_x.shape)

In [ ]:
def preprocess_dataset(data):
    mfccs = []
    for i in data:
        extracted_features = librosa.feature.mfcc(y=i,
                                              sr=16000,
                                              n_mfcc=40)
        mfccs.append(extracted_features)

    return mfccs

In [ ]:
train_mfccs = preprocess_dataset(train_x)
train_mfccs = np.array(train_mfccs)
train_mfccs = train_mfccs.reshape(-1, train_mfccs.shape[1], train_mfccs.shape[2], 1)

In [ ]:
_mfccs = preprocess_dataset(test_x)
train_mfccs = np.array(train_mfccs)
train_mfccs = train_mfccs.reshape(-1, train_mfccs.shape[1], train_mfccs.shape[2], 1)

In [ ]:
import torchvision.datasets as datasets # 데이터셋 집합체
import torchvision.transforms as transforms # 변환 툴

from torch.utils.data import DataLoader # 학습 및 배치로 모델에 넣어주기 위한 툴
from torch.utils.data import DataLoader, Dataset

class CustomDataset(Dataset):
    def __init__(self, X, y, train_mode=True, transforms=None): #필요한 변수들을 선언
        self.X = X
        self.y = y
        self.train_mode = train_mode
        self.transforms = transforms

    def __getitem__(self, index): #index번째 data를 return
        X = self.X[index]

        if self.transforms is not None:
            X = self.transforms(X)

        if self.train_mode:
            y = self.y[index]
            return X, y
        else:
            return X

    def __len__(self): #길이 return
        return len(self.X)

In [ ]:
train_x = train_mfccs
vali_x = test_mfccs
train_y  = train_y
vali_y = test_y

In [ ]:
len(train_x),len(vali_x),len(train_y),len(vali_y)

In [ ]:
# 에포크 설정
num_epochs = 100

# 배치 사이즈 설정
batch_size = 5

#만든 train dataset를 DataLoader에 넣어 batch 만들기
train_dataset = CustomDataset(X=train_x, y=train_y)
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)

In [ ]:
vali_dataset = CustomDataset(X=vali_x, y=vali_y)
vali_loader = DataLoader(vali_dataset, batch_size = batch_size, shuffle=False)

In [ ]:
train_batches = len(train_loader)
vali_batches = len(vali_loader)

print('/ total train batches :', train_batches)
print('/ total valid batches :', vali_batches)

In [ ]:
import torch.nn as nn # 신경망들이 포함됨

class CNNclassification(torch.nn.Module):
    def __init__(self):
        super(CNNclassification, self).__init__()
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(40, 10, kernel_size=(3,3), stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=(5,5))) #pooling layer

        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(10, 100, kernel_size=(3,3), stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=(5,5))) #pooling layer

        self.layer3 = torch.nn.Sequential(
            nn.Conv2d(100, 200, kernel_size=(3,3), stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=(5,5))) #pooling layer

        self.layer4 = torch.nn.Sequential(
            nn.Conv2d(200, 300, kernel_size=(3,3), stride=1, padding=1), #cnn layer
            nn.ReLU(), #activation function
            nn.MaxPool2d(kernel_size=(5,5))) #pooling layer

        self.fc_layer = nn.Sequential(
            nn.Linear(48600, 10) #fully connected layer(ouput layer)
        )

    def forward(self, x):

        x = self.layer1(x) #1층

        x = self.layer2(x) #2층

        x = self.layer3(x) #3층

        x = self.layer4(x) #4층

        x = torch.flatten(x, start_dim=1) # N차원 배열 -> 1차원 배열

        out = self.fc_layer(x)
        return out

In [ ]:
import torch.optim as optim

In [ ]:
import torch.optim as optim # 최적화 알고리즘들이 포함힘

model = CNNclassification().to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(params = model.parameters(), lr = 1e-3 )
scheduler = None

In [ ]:
from tqdm.auto import tqdm

def train(model, optimizer, train_loader, scheduler, device):
    model.to(device)
    n = len(train_loader)
    best_acc = 0

    for epoch in range(1,num_epochs): #에포크 설정
        model.train() #모델 학습
        running_loss = 0.0

        for wav, label in tqdm(iter(train_loader)):

            wav, label = wav.to(device), label.to(device) #배치 데이터
            optimizer.zero_grad() #배치마다 optimizer 초기화

            # Data -> Model -> Output
            logit = model(wav) #예측값 산출
            loss = criterion(logit, label) #손실함수 계산

            # 역전파
            loss.backward() #손실함수 기준 역전파
            optimizer.step() #가중치 최적화
            running_loss += loss.item()

        print('[%d] Train loss: %.10f' %(epoch, running_loss / len(train_loader)))

        if scheduler is not None:
            scheduler.step()


        #Validation set 평가
        model.eval() #evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
        vali_loss = 0.0
        correct = 0

        with torch.no_grad(): #파라미터 업데이트 안하기 때문에 no_grad 사용
            for wav, label in tqdm(iter(vali_loader)):

                wav, label = wav.to(device), label.to(device)
                logit = model(wav)
                vali_loss += criterion(logit, label)
                pred = logit.argmax(dim=1, keepdim=True)  #10개의 class중 가장 값이 높은 것을 예측 label로 추출
                correct += pred.eq(label.view_as(pred)).sum().item() #예측값과 실제값이 맞으면 1 아니면 0으로 합산
        vali_acc = 100 * correct / len(vali_loader.dataset)
        print('Vail set: Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(vali_loss / len(vali_loader), correct, len(vali_loader.dataset), 100 * correct / len(vali_loader.dataset)))

        #베스트 모델 저장
        if best_acc < vali_acc:
            best_acc = vali_acc
            torch.save(model.state_dict(), './best_model2.pth') #이 디렉토리에 best_model.pth을 저장
            print('Model Saved.')

In [ ]:
train(model, optimizer, train_loader, scheduler, device)

# 음악 특성 피처 모델

In [ ]:
spec_df = pd.read_csv('/content/drive/MyDrive/dataton/spectral_30.csv')
spec_df.shape, spec_df.columns

((3259, 56),
 Index(['name', 'target', 'rolloff_mean', 'rolloff_var', 'centroid_mean',
        'centroid_var', 'flux_mean', 'flux_var', 'contrast_mean',
        'contrast_var', 'flatness_mean', 'flatness_var', 'bw_mean', 'bw_var',
        'chroma_mean', 'chroma_var', 'mfcc1_mean', 'mfcc1_var', 'mfcc2_mean',
        'mfcc2_var', 'mfcc3_mean', 'mfcc3_var', 'mfcc4_mean', 'mfcc4_var',
        'mfcc5_mean', 'mfcc5_var', 'mfcc6_mean', 'mfcc6_var', 'mfcc7_mean',
        'mfcc7_var', 'mfcc8_mean', 'mfcc8_var', 'mfcc9_mean', 'mfcc9_var',
        'mfcc10_mean', 'mfcc10_var', 'mfcc11_mean', 'mfcc11_var', 'mfcc12_mean',
        'mfcc12_var', 'mfcc13_mean', 'mfcc13_var', 'mfcc14_mean', 'mfcc14_var',
        'mfcc15_mean', 'mfcc15_var', 'mfcc16_mean', 'mfcc16_var', 'mfcc17_mean',
        'mfcc17_var', 'mfcc18_mean', 'mfcc18_var', 'mfcc19_mean', 'mfcc19_var',
        'mfcc20_mean', 'mfcc20_var'],
       dtype='object'))

## 기초 모델

In [ ]:
X = spec_df.drop('name', axis = 1).iloc[:,1:]
y = spec_df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size = 0.1, stratify=y, random_state=100)
model = ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=20, max_features='sqrt',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      monotonic_cst=None, n_estimators=400, n_jobs=-1,
                      oob_score=False, random_state=100, verbose=0,
                      warm_start=False)
model.fit(X_train, y_train)
print("훈련 세트 정확도: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(model.score(X_test, y_test)))

# 정규화

In [ ]:
X = spec_df.drop('name', axis = 1).iloc[:,1:]

## minmax

In [ ]:
scaler = MinMaxScaler()
minmax = scaler.fit_transform(X)
minmax_df = pd.DataFrame(minmax, columns = X.columns)

In [ ]:
X = minmax_df
y = spec_df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size = 0.3, stratify=y, random_state=100)

model = ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=20, max_features='sqrt',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      monotonic_cst=None, n_estimators=400, n_jobs=-1,
                      oob_score=False, random_state=100, verbose=0,
                      warm_start=False)
# scores = cross_validate(model, X_train, y_train, return_train_score = True, n_jobs = -1)
# print(np.mean(scores['train_score']),np.mean(scores['test_score']))
model.fit(X_train, y_train)
print("훈련 세트 정확도: {:.3f}".format(model.score(X_train, y_train)))
print("테스트 세트 정확도: {:.3f}".format(model.score(X_test, y_test)))

훈련 세트 정확도: 0.982
테스트 세트 정확도: 0.536


## std

In [ ]:
scaler = StandardScaler()
std_sc = scaler.fit_transform(X)
std_df = pd.DataFrame(std_sc, columns = X.columns)

In [ ]:
X = std_df
y = spec_df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size = 0.3, stratify=y, random_state=100)
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_validate
model = ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=20, max_features='sqrt',
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      monotonic_cst=None, n_estimators=400, n_jobs=-1,
                      oob_score=False, random_state=100, verbose=0,
                      warm_start=False)
scores = cross_validate(model, X_train, y_train, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))
# model.fit(X_train, y_train)
# print("훈련 세트 정확도: {:.3f}".format(model.score(X_train, y_train)))
# print("테스트 세트 정확도: {:.3f}".format(model.score(X_test, y_test)))

0.9853132059601057 0.5251948251372414


##그리드 서치 진행

In [ ]:
X = spec_df.drop('name', axis = 1).iloc[:,1:]
y = spec_df['target']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size = 0.3, stratify=y)
model = ExtraTreesClassifier()
# 하이퍼파라미터 튜닝을 위한 그리드 서치
param_grid = {
    'n_estimators': [400,500,600],
    'max_depth': [30,40],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 최적의 파라미터로 모델 훈련
best_model = grid_search.best_estimator_

# 예측
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

# 정확도 계산
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"최적의 파라미터: {grid_search.best_params_}")
print(f"훈련 세트 정확도: {train_accuracy:.3f}")
print(f"테스트 세트 정확도: {test_accuracy:.3f}")

Fitting 5 folds for each of 24 candidates, totalling 120 fits
최적의 파라미터: {'max_depth': 40, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 400}
훈련 세트 정확도: 0.985
테스트 세트 정확도: 0.535


# auto ML

In [ ]:
X = spec_df.drop('name', axis = 1).iloc[:,1:]

y = spec_df['target']
scaler = MinMaxScaler()

minmax = scaler.fit_transform(X)
minmax_df = pd.DataFrame(minmax, columns = X.columns)
X['target'] = spec_df['target']

In [ ]:
clf = setup(data = X, target='target', train_size = 0.8, verbose = True, data_split_shuffle=True,session_id= 100)

In [ ]:
top_5_model = compare_models(fold=10, round=3, sort="Accuracy", n_select=5)

In [ ]:
blend3 = blend_models(estimator_list = top_5_model, round =4, choose_better=True, optimize='Accuracy')

In [ ]:
final_model = finalize_model(blend3)

In [ ]:
prediction = predict_model(final_model, data = test_df.iloc[:,1:])